In [ ]:
import pandas as pd
import os
import numpy as np
# import fastf1
from keras.preprocessing.sequence import TimeseriesGenerator
pd.set_option('display.max_columns', None)
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.preprocessing import MultiLabelBinarizer

In [ ]:
df_complete = pd.read_csv("complete_data.csv")

<ipython-input-11-c2e2fc0e7eb3>:1: DtypeWarning: Columns (28) have mixed types. Specify dtype option on import or set low_memory=False.
  df_complete = pd.read_csv("complete_data.csv")


# EDA

In [ ]:
print(df_complete.shape)

(136122, 39)


In [ ]:
df_complete.head()

,Time,Driver,DriverNumber,LapTime,LapNumber,Stint,PitOutTime,PitInTime,Sector1Time,Sector2Time,Sector3Time,Sector1SessionTime,Sector2SessionTime,Sector3SessionTime,SpeedI1,SpeedI2,SpeedFL,SpeedST,IsPersonalBest,Compound,TyreLife,FreshTyre,Team,LapStartTime,LapStartDate,TrackStatus,Position,Deleted,DeletedReason,FastF1Generated,IsAccurate,uid,AirTemp,Humidity,Pressure,Rainfall,TrackTemp,WindDirection,WindSpeed
0,0 days 00:08:53.241000,GAS,10,0 days 00:01:45.060000,1.0,NaN,NaN,NaN,NaN,0 days 00:00:25.495000,0 days 00:00:37.955000,NaN,0 days 00:08:15.609000,0 days 00:08:53.829000,257.0,283.0,286.0,237.0,False,NaN,NaN,True,Toro Rosso,0 days 00:07:07.988000,2018-03-25 05:13:19.169,1.0,17.0,False,NaN,False,False,2018_1,24.2,36.3,997.0,False,38.9,307,4.1
1,0 days 00:10:26.613000,GAS,10,0 days 00:01:33.372000,2.0,1.0,NaN,NaN,0 days 00:00:31.357000,0 days 00:00:24.825000,0 days 00:00:37.190000,0 days 00:09:24.632000,0 days 00:09:49.457000,0 days 00:10:26.647000,266.0,286.0,277.0,270.0,True,ULTRASOFT,1.0,True,Toro Rosso,0 days 00:08:53.241000,2018-03-25 05:15:04.422,1.0,17.0,False,NaN,False,True,2018_1,24.2,36.3,996.9,False,38.2,296,3.8
2,0 days 00:11:59.474000,GAS,10,0 days 00:01:32.861000,3.0,1.0,NaN,NaN,0 days 00:00:31.160000,0 days 00:00:24.725000,0 days 00:00:36.976000,0 days 00:10:57.807000,0 days 00:11:22.532000,0 days 00:11:59.508000,260.0,283.0,294.0,277.0,True,ULTRASOFT,2.0,True,Toro Rosso,0 days 00:10:26.613000,2018-03-25 05:16:37.794,1.0,17.0,False,NaN,False,True,2018_1,23.9,36.5,997.1,False,36.7,289,4.3
3,0 days 00:13:31.658000,GAS,10,0 days 00:01:32.184000,4.0,1.0,NaN,NaN,0 days 00:00:30.835000,0 days 00:00:24.730000,0 days 00:00:36.619000,0 days 00:12:30.343000,0 days 00:12:55.073000,0 days 00:13:31.692000,264.0,283.0,294.0,296.0,True,ULTRASOFT,3.0,True,Toro Rosso,0 days 00:11:59.474000,2018-03-25 05:18:10.655,1.0,17.0,False,NaN,False,True,2018_1,23.8,35.7,997.1,False,36.6,210,2.7
4,0 days 00:15:03.990000,GAS,10,0 days 00:01:32.332000,5.0,1.0,NaN,NaN,0 days 00:00:30.716000,0 days 00:00:24.821000,0 days 00:00:36.795000,0 days 00:14:02.408000,0 days 00:14:27.229000,0 days 00:15:04.024000,266.0,283.0,283.0,295.0,False,ULTRASOFT,4.0,True,Toro Rosso,0 days 00:13:31.658000,2018-03-25 05:19:42.839,2.0,17.0,False,NaN,False,True,2018_1,23.5,36.3,997.2,False,36.4,267,2.5


# Pre-processing

Delete all first laps

In [ ]:
# Delete all first laps
df_complete = df_complete[df_complete["LapNumber"]!= 1.0]

Feature selection

In [ ]:
# Select the variables that we will be using --> zie notities pre-processing
df_complete = df_complete[["uid", "Driver", "LapTime", "Stint", "PitOutTime", "PitInTime", "Compound", "TyreLife", "Team", "TrackStatus", "Position",
                           "AirTemp", "Humidity", "Pressure", "Rainfall", "TrackTemp", "WindDirection", "WindSpeed"]]

print(df_complete.shape)

(133627, 18)


Feature creation - Laptype

In [ ]:
def get_lap_type(row):
    if pd.notnull(row['PitInTime']):
        return 'InLap'
    elif pd.notnull(row['PitOutTime']):
        return 'Outlap'
    else:
        return 'Lap'

In [ ]:
# Apply the function to create new column "LapType"
df_complete['LapType'] = df_complete.apply(get_lap_type, axis=1)
df_complete.drop(["PitOutTime", "PitInTime"], axis=1, inplace=True)

Missing values inspection

In [ ]:
df_complete.isna().sum().sum()

2941

Lap Time pre-processing

In [ ]:
# Make LapTime of type timedelta
df_complete["LapTime"] = pd.to_timedelta(df_complete["LapTime"])

In [ ]:
# Custom function to convert timedelta to milliseconds
def convert_timedelta_to_milliseconds(td):
    return td.total_seconds() * 1000 + td.microseconds / 1000

# Apply the custom function to the 'lap_time' columm
df_complete['LapTime'] = df_complete['LapTime'].apply(convert_timedelta_to_milliseconds)

Make distinction between rain and no rain races

In [ ]:
# Group by 'uid' and check if any lap had rainfall
rainfall_races = df_complete.groupby('uid')['Rainfall'].any()

# Extract uids where rainfall was present
rainfall_uids = rainfall_races[rainfall_races].index

# Extract uids where no rainfall was present
no_rainfall_uids = rainfall_races[~rainfall_races].index

# Dataframe with races where rainfall was present
rainfall_df = df_complete[df_complete['uid'].isin(rainfall_uids)]

# Dataframe with races where no rainfall was present
no_rainfall_df = df_complete[df_complete['uid'].isin(no_rainfall_uids)]

In [ ]:
print(rainfall_df.shape)
print(no_rainfall_df.shape)

(18808, 17)
(114819, 17)


create windows - univariate - NO rain

In [ ]:
# Group by "uid" and "Driver"
no_rainfall_df = no_rainfall_df.groupby(["uid", "Driver"])

# Initialize an empty array to append the resulting arrays

total_input_windows_u = np.empty((0, 10))
total_output_windows_u = np.empty((0, 10))

# Iterate over each group and convert to sliding windows
for (uid, driver), groupu in no_rainfall_df:

   input_seq_u = groupu["LapTime"].to_list()

   # Make sure that sequences that have less than 20 data points are not included
   if len(input_seq_u) < 20:
      continue

   output_multi_u = pd.concat([groupu["LapTime"], groupu["LapTime"].shift(-1), groupu["LapTime"].shift(-2),
                          groupu["LapTime"].shift(-3), groupu["LapTime"].shift(-4), groupu["LapTime"].shift(-5), groupu["LapTime"].shift(-6),
                          groupu["LapTime"].shift(-7), groupu["LapTime"].shift(-8), groupu["LapTime"].shift(-9)], axis=1).to_numpy().tolist()

   ts_generator_u = TimeseriesGenerator(input_seq_u, output_multi_u, length = 10)

   input_windows_u = ts_generator_u[0][0][:-9] # --> [:-4] because the last n widows (n = how many shifts are executed) are not complete (non existent)
   output_windows_u = ts_generator_u[0][1][:-9]

   total_input_windows_u = np.vstack((total_input_windows_u, input_windows_u))
   total_output_windows_u = np.vstack((total_output_windows_u, output_windows_u))

In [ ]:
print(total_output_windows_u.shape)
print(total_input_windows_u.shape)

(76102, 10)
(76102, 10)


create windows - univariate - rain

In [ ]:
# Group by "uid" and "Driver"
rainfall_df = rainfall_df.groupby(["uid", "Driver"])

# Initialize an empty array to append the resulting arrays

total_input_windows_u_r = np.empty((0, 10))
total_output_windows_u_r = np.empty((0, 10))

# Iterate over each group and convert to sliding windows
for (uid, driver), groupu in rainfall_df:

   input_seq_u = groupu["LapTime"].to_list()

   # Make sure that sequences that have less than 20 data points are not included
   if len(input_seq_u) < 20:
      continue

   output_multi_u = pd.concat([groupu["LapTime"], groupu["LapTime"].shift(-1), groupu["LapTime"].shift(-2),
                          groupu["LapTime"].shift(-3), groupu["LapTime"].shift(-4), groupu["LapTime"].shift(-5), groupu["LapTime"].shift(-6),
                          groupu["LapTime"].shift(-7), groupu["LapTime"].shift(-8), groupu["LapTime"].shift(-9)], axis=1).to_numpy().tolist()

   ts_generator_u = TimeseriesGenerator(input_seq_u, output_multi_u, length = 10)

   input_windows_u = ts_generator_u[0][0][:-9] # --> [:-4] because the last n widows (n = how many shifts are executed) are not complete (non existent)
   output_windows_u = ts_generator_u[0][1][:-9]

   total_input_windows_u_r = np.vstack((total_input_windows_u_r, input_windows_u))
   total_output_windows_u_r = np.vstack((total_output_windows_u_r, output_windows_u))

In [ ]:
print(total_output_windows_u_r.shape)
print(total_input_windows_u_r.shape)

(12655, 10)
(12655, 10)


Create the windows - multivariate NO rain

In [ ]:
# Group by "uid" and "Driver"
no_rainfall_df = no_rainfall_df.groupby(["uid", "Driver"])

# Initialize an empty array to append the resulting arrays

total_input_windows_m = np.empty((0, 10, 3))
total_output_windows_m = np.empty((0,10))

# Iterate over each group and convert to sliding windows
for (uid, driver), groupm in no_rainfall_df:

   # THIS HAS TO BE ADAPTED TO THE ENCODINGS
   input_seq_m = groupm[["LapTime", "TrackStatus", "Position"]].to_numpy().tolist()

   # Make sure that sequences that have less than 20 data points are not included
   if len(input_seq_m) < 20:
      continue

   output_multi_m = pd.concat([groupm["LapTime"], groupm["LapTime"].shift(-1), groupm["LapTime"].shift(-2),
                          groupm["LapTime"].shift(-3), groupm["LapTime"].shift(-4), groupm["LapTime"].shift(-5), groupm["LapTime"].shift(-6),
                          groupm["LapTime"].shift(-7), groupm["LapTime"].shift(-8), groupm["LapTime"].shift(-9)], axis=1).to_numpy().tolist()

   ts_generator_m = TimeseriesGenerator(input_seq_m, output_multi_m, length = 10)

   input_windows_m = ts_generator_m[0][0][:-9] # --> [:-4] because the last n widows (n = how many shifts are executed) are not complete (non existent)
   output_windows_m = ts_generator_m[0][1][:-9]

   total_input_windows_m = np.vstack((total_input_windows_m, input_windows_m))
   total_output_windows_m = np.vstack((total_output_windows_m, output_windows_m))

In [ ]:
print(total_output_windows_m.shape)
print(total_input_windows_m.shape)

(76102, 10)
(76102, 10, 3)


Create the windows - multivariate rain

In [ ]:
# Group by "uid" and "Driver"
rainfall_df = rainfall_df.groupby(["uid", "Driver"])

# Initialize an empty array to append the resulting arrays

total_input_windows_m_r = np.empty((0, 10, 3))
total_output_windows_m_r = np.empty((0,10))

# Iterate over each group and convert to sliding windows
for (uid, driver), groupm in rainfall_df:

   # THIS HAS TO BE ADAPTED TO THE ENCODINGS
   input_seq_m = groupm[["LapTime", "TrackStatus", "Position"]].to_numpy().tolist()

   # Make sure that sequences that have less than 20 data points are not included
   if len(input_seq_m) < 20:
      continue

   output_multi_m = pd.concat([groupm["LapTime"], groupm["LapTime"].shift(-1), groupm["LapTime"].shift(-2),
                          groupm["LapTime"].shift(-3), groupm["LapTime"].shift(-4), groupm["LapTime"].shift(-5), groupm["LapTime"].shift(-6),
                          groupm["LapTime"].shift(-7), groupm["LapTime"].shift(-8), groupm["LapTime"].shift(-9)], axis=1).to_numpy().tolist()

   ts_generator_m = TimeseriesGenerator(input_seq_m, output_multi_m, length = 10)

   input_windows_m = ts_generator_m[0][0][:-9] # --> [:-4] because the last n widows (n = how many shifts are executed) are not complete (non existent)
   output_windows_m = ts_generator_m[0][1][:-9]

   total_input_windows_m_r = np.vstack((total_input_windows_m_r, input_windows_m))
   total_output_windows_m_r = np.vstack((total_output_windows_m_r, output_windows_m))

In [ ]:
print(total_output_windows_m_r.shape)
print(total_input_windows_m_r.shape)

(12655, 10)
(12655, 10, 3)


Dealing with missing values for laptimes - individual missing

In [ ]:
# Univariate windows - No rain
df_uni_input = pd.DataFrame(total_input_windows_u)
df_uni_input.fillna(axis=1, method='backfill', inplace=True)
df_uni_input.fillna(axis=1, method='ffill', inplace=True)
input_windows_uni = df_uni_input.to_numpy()

df_uni_output = pd.DataFrame(total_input_windows_u)
df_uni_output.fillna(axis=1, method='backfill', inplace=True)
df_uni_output.fillna(axis=1, method='ffill', inplace=True)
output_windows_uni = df_uni_output.to_numpy()

In [ ]:
print(input_windows_uni.shape)
print(output_windows_uni.shape)

(76102, 10)
(76102, 10)


In [ ]:
# Univariate windows - Rain
df_uni_input = pd.DataFrame(total_input_windows_u_r)
df_uni_input.fillna(axis=1, method='backfill', inplace=True)
df_uni_input.fillna(axis=1, method='ffill', inplace=True)
input_windows_uni_r = df_uni_input.to_numpy()

df_uni_output = pd.DataFrame(total_input_windows_u_r)
df_uni_output.fillna(axis=1, method='backfill', inplace=True)
df_uni_output.fillna(axis=1, method='ffill', inplace=True)
output_windows_uni_r = df_uni_output.to_numpy()

In [ ]:
print(input_windows_uni_r.shape)
print(output_windows_uni_r.shape)

(12655, 10)
(12655, 10)


replace multi lap times with uni laptimes for effcient missing values handling

In [ ]:
input_windows_uni_reshaped = input_windows_uni[:, :, np.newaxis]

In [ ]:
input_windows_multi = np.concatenate((input_windows_uni_reshaped, total_input_windows_m),
                                     axis=2)
input_windows_multi = np.delete(input_windows_multi, 1, axis=2)

In [ ]:
print(input_windows_multi.shape)

(76102, 10, 3)


In [ ]:
input_windows_uni_r_reshaped = input_windows_uni_r[:, :, np.newaxis]

In [ ]:
input_windows_multi_r = np.concatenate((input_windows_uni_r_reshaped, total_input_windows_m_r),
                                     axis=2)
input_windows_multi_r = np.delete(input_windows_multi_r, 1, axis=2)

In [ ]:
print(input_windows_multi_r.shape)

(12655, 10, 3)


Encoding

In [ ]:
# Define column indices for numerical and categorical columns
numerical_columns = [0,2]
track_status_column = [1]

# Separate numerical and categorical data using column indices
numerical_data = input_windows_multi[:, :, numerical_columns].reshape(-1, len(numerical_columns))
track_status = input_windows_multi[:, :, track_status_column].reshape(-1, len(track_status_column))

In [ ]:
# Separate numerical and categorical data using column indices
numerical_data_r = input_windows_multi_r[:, :, numerical_columns].reshape(-1, len(numerical_columns))
track_status_r = input_windows_multi_r[:, :, track_status_column].reshape(-1, len(track_status_column))

In [ ]:
print(numerical_data.shape)
print(track_status.shape)
print(numerical_data_r.shape)
print(track_status_r.shape)

(761020, 2)
(761020, 1)
(126550, 2)
(126550, 1)


In [ ]:
# Encoding trackstatus - No rain

# Get the track status code in the right formnat
track_status = track_status.astype(float)
track_status = track_status.astype(int)
track_status = track_status.astype(str)

# Transform each element into a tuple of characters
track_status_res = np.array([tuple(val[0]) for val in track_status], dtype=object)

mlb = MultiLabelBinarizer()
track_status_encoded = mlb.fit_transform(track_status_res)
mlb.classes_

array(['1', '2', '4', '5', '6', '7'], dtype=object)

In [ ]:
# Encoding trackstatus - rain

# Get the track status code in the right formnat
track_status_r = track_status_r.astype(float)
track_status_r = track_status_r.astype(int)
track_status_r = track_status_r.astype(str)

# Transform each element into a tuple of characters
track_status_resh = np.array([tuple(val[0]) for val in track_status_r], dtype=object)

mlb = MultiLabelBinarizer()
track_status_encoded_r = mlb.fit_transform(track_status_resh)
mlb.classes_

array(['1', '2', '4', '5', '6', '7'], dtype=object)

In [ ]:
print(numerical_data.shape)
print(track_status_encoded.shape)

(761020, 2)
(761020, 6)


In [ ]:
print(numerical_data_r.shape)
print(track_status_encoded_r.shape)

(126550, 2)
(126550, 6)


In [ ]:
numerical_data = numerical_data.astype(float)
track_status_encoded = track_status_encoded.astype(float)
numerical_data_r = numerical_data_r.astype(float)
track_status_encoded_r = track_status_encoded_r.astype(float)

In [ ]:
# Concatenate numerical and encoded categorical data along axis=1
encoded_data = np.concatenate((numerical_data, track_status_encoded), axis=1)
encoded_data_r = np.concatenate((numerical_data_r, track_status_encoded_r), axis=1)

In [ ]:
print(encoded_data.shape)
print(encoded_data_r.shape)

(761020, 8)
(126550, 8)


In [ ]:
input_windows_multi_encoded = encoded_data.reshape(76102, 10, encoded_data.shape[1])
print(input_windows_multi_encoded.shape)

(76102, 10, 8)


In [ ]:
input_windows_multi_encoded_r = encoded_data_r.reshape(12655, 10, encoded_data_r.shape[1])
print(input_windows_multi_encoded_r.shape)

(12655, 10, 8)


Normalising

In [ ]:
print(input_windows_uni.shape)
print(input_windows_uni_r.shape)
print()
print(input_windows_multi_encoded.shape)
print(input_windows_multi_encoded_r.shape)

(76102, 10)
(12655, 10)

(76102, 10, 8)
(12655, 10, 8)


In [ ]:
print(output_windows_uni.shape)
print(output_windows_uni_r.shape)

(76102, 10)
(12655, 10)


In [ ]:
# univaroates
scaler = StandardScaler()

input_windows_uni = scaler.fit_transform(input_windows_uni)
input_windows_uni_r = scaler.fit_transform(input_windows_uni_r)

In [ ]:
# multivariate
numericals = input_windows_multi_encoded[:,:,:2]
categoricals = input_windows_multi_encoded[:,:,2:]
print(numericals.shape)
print(categoricals.shape)

(76102, 10, 2)
(76102, 10, 6)


In [ ]:
# Standardize only numerical values
scaler = StandardScaler()

# Reshape the multi input nto a 2D array
num_samples, num_timesteps, num_features = numericals.shape
numericals_2d = numericals.reshape(num_samples * num_timesteps, num_features)

numericals_std_2d = scaler.fit_transform(numericals_2d)

# Reshape multi input back to 3D array
numericals_std = numericals_std_2d.reshape(num_samples, num_timesteps, num_features)

In [ ]:
# concatenate categoricals and numericals_std
input_windows_multi =  np.concatenate((numericals_std, categoricals), axis=2)

In [ ]:
numericals_r = input_windows_multi_encoded_r[:,:,:2]
categoricals_r = input_windows_multi_encoded_r[:,:,2:]
print(numericals_r.shape)
print(categoricals_r.shape)

(12655, 10, 2)
(12655, 10, 6)


In [ ]:
# Normalise all values
scaler = StandardScaler()

# Standardize alleen de numerical values

# Reshape the multi input nto a 2D array
num_samples, num_timesteps, num_features = numericals_r.shape
numericals_2d = numericals_r.reshape(num_samples * num_timesteps, num_features)

numericals_std_2d = scaler.fit_transform(numericals_2d)

# Reshape multi input back to 3D array
numericals_std = numericals_std_2d.reshape(num_samples, num_timesteps, num_features)

In [ ]:
# concatenate categoricals and numericals_std
input_windows_multi_r =  np.concatenate((numericals_std, categoricals_r), axis=2)

# Check and save

In [ ]:
print(input_windows_uni.shape)
print(input_windows_uni_r.shape)

(76102, 10)
(12655, 10)


In [ ]:
print(input_windows_multi.shape)
print(input_windows_multi_r.shape)

(76102, 10, 8)
(12655, 10, 8)


In [ ]:
print(output_windows_uni.shape)
print(output_windows_uni_r.shape)

(76102, 10)
(12655, 10)


In [ ]:
np.save("input_windows_uni", input_windows_uni)

In [ ]:
np.save("input_windows_uni_r", input_windows_uni_r)

In [ ]:
np.save("input_windows_multi", input_windows_multi)

In [ ]:
np.save("input_windows_multi_r", input_windows_multi_r)

In [ ]:
np.save("output_windows_uni", output_windows_uni)

In [ ]:
np.save("output_windows_uni_r", output_windows_uni_r)